In [3]:
import os
import sys
sys.dont_write_bytecode = True
module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)
from transformers import (
    AutoTokenizer,
    BertTokenizer,
    BartTokenizer,
    T5Tokenizer,
    GPT2Tokenizer,
)
from typing import Union, List, Dict, Tuple
from datasets import load_dataset, Dataset
import pandas as pd
from tqdm import tqdm
import torch
import torch.nn as nn
import torchtext
import torch.nn.functional as F

2023-03-12 16:52:19.848632: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 16:52:20.695368: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-12 16:52:20.695445: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-12 16:52:20.695456: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [8]:
from itertools import chain
from typing import List, Optional
from transformers import AutoTokenizer
from transformers.testing_utils import CaptureLogger
from transformers.utils.logging import get_logger


class DGDataset:
    def __init__(
        self, 
        dataset: str = "blended_skill_talk",
        task: str = "seq2seq",
        tokenizer: AutoTokenizer = None,
        max_source_length: int = 512,
        max_target_length: int = 512,
        padding: str = "max_length",
        ignore_pad_token_for_loss: bool = True,
        preprocessing_num_workers: int = None,
        overwrite_cache: bool = True,
    ):
        self.dataset = dataset
        self.task = task
        self.tokenizer = tokenizer
        self.max_source_length = max_source_length
        self.max_target_length = max_target_length
        self.padding = padding
        self.ignore_pad_token_for_loss = ignore_pad_token_for_loss
        self.preprocessing_num_workers = preprocessing_num_workers
        self.overwrite_cache = overwrite_cache
        # since this will be pickled to avoid _LazyModule error in Hasher force logger loading before tokenize_function
        self.tok_logger = get_logger("transformers.tokenization_utils_base")


    def prepare_context(self, instance: dict):
        if self.dataset == 'blended_skill_talk':
            num_entries = len(instance["free_messages"])
            total_entries = num_entries
            if self.task == 'seq2seq':
                persona_pieces = f"<PS>{instance['personas'][1]}"
                if instance['context'] == "wizard_of_wikipedia":
                    additional_context_pieces = f"<CTX>{instance['additional_context']}."
                else:
                    additional_context_pieces = ""
                context = persona_pieces + additional_context_pieces
            else:
                num_entries = min(num_entries, 2)
                context = ''
            prev_utt_pc = [sent for sent in instance["previous_utterance"] if sent != '']

        elif self.dataset == 'conv_ai_2':
            total_entries = len(instance['dialog'])
            num_entries = total_entries//2
            if self.task == 'seq2seq':
                user_profile = ' '.join([''.join(x) for x in instance['user_profile']])
                persona_pieces = f"<PS>{user_profile}"
                context = persona_pieces
            else:
                num_entries = min(num_entries, 2)
                context = ''
            prev_utt_pc = []

        elif self.dataset == 'empathetic_dialogues':
            total_entries = len(instance['dialog'])
            num_entries = total_entries//2
            if self.task == 'seq2seq':
                persona_pieces = f"<PS>{instance['prompt']}"
                additional_context_pieces = f"<CTX>{instance['context']}."
                context = persona_pieces + additional_context_pieces
            else:
                num_entries = min(num_entries, 2)
                context = ''
            prev_utt_pc = []

        elif self.dataset == 'AlekseyKorshuk/persona-chat':
            total_entries = len(instance['utterances'])
            num_entries = total_entries//2
            if self.task == 'seq2seq':
                user_profile = ' '.join(instance['personality'])
                persona_pieces = f"<PS>{user_profile}"
                context = persona_pieces
            else:
                num_entries = min(num_entries, 2)
                context = ''
            prev_utt_pc = []

        else:
            raise ValueError("Dataset not supported.")
        return num_entries, total_entries, context, prev_utt_pc


    def prepare_entry(
        self, 
        instance: dict, 
        entry_idx: int, 
        context: str, 
        prev_utt_pc: List[str], 
        total_entries: int,
    ):
        if self.dataset == 'blended_skill_talk':
            free_message = instance['free_messages'][entry_idx]
            guided_message = instance['guided_messages'][entry_idx]
            references = [values[entry_idx] for key, values in instance['suggestions'].items()]

        elif self.dataset == 'conv_ai_2':
            free_message = instance['dialog'][entry_idx*2]['text']
            if entry_idx*2+1 >= total_entries:
                guided_message = None
            else:
                guided_message = instance['dialog'][entry_idx*2+1]['text']
            references = []

        elif self.dataset == 'empathetic_dialogues':
            free_message = instance['dialog'][entry_idx*2]['text']
            if entry_idx*2+1 >= total_entries:
                guided_message = None
            else:
                guided_message = instance['dialog'][entry_idx*2+1]['text']
            references = []

        elif self.dataset == 'AlekseyKorshuk/persona-chat':
            free_message = instance['utterances'][entry_idx*2]['history'][-1]
            if entry_idx*2+1 >= total_entries:
                guided_message = None
            else:
                guided_message = instance['utterances'][entry_idx*2+1]['history'][-1]
            references = instance['utterances'][entry_idx*2]['candidates']
            
        else:
            raise ValueError("Dataset not supported.")

        if not prev_utt_pc:
            original_context = context
        else:
            sp_token = '<SEP>' if self.task == 'seq2seq' else ' '
            original_context = context + sp_token + sp_token.join(prev_utt_pc)
        
        references.append(guided_message)
        return free_message, guided_message, original_context, references


    def tokenize_and_align_labels(self, instance: dict):
        num_entries, total_entries, context, prev_utt_pc = self.prepare_context(instance)
        inputs, labels = [], []
        for entry_idx in range(num_entries):
            free_message, guided_message, original_context, references = self.prepare_entry(
                instance, 
                entry_idx, 
                context, 
                prev_utt_pc,
                total_entries,
            )
            if guided_message is None:
                continue
            # Input & Output
            if self.task == 'seq2seq':
                text = original_context + self.tokenizer.eos_token + free_message
            else:
                text = original_context + free_message + guided_message

            inputs.append(text)
            labels.append(guided_message)
            prev_utt_pc += [
                free_message,
                guided_message,
            ]
        
        if not inputs:
            return {"input_ids": [], "labels": [], "attention_mask": []}

        if self.task == 'seq2seq':
            inputs = self.tokenizer(inputs, max_length=self.max_source_length, padding=self.padding, truncation=True)
            # Setup the tokenizer for targets
            with self.tokenizer.as_target_tokenizer():
                labels = self.tokenizer(labels, max_length=self.max_target_length, padding=self.padding, truncation=True)
            
            # If we are padding here, replace all tokenizer.pad_token_id in the labels by -100 
            # when we want to ignore padding in the loss.
            if self.padding == "max_length" and self.ignore_pad_token_for_loss:
                labels["input_ids"] = [
                    [(l if l != self.tokenizer.pad_token_id else -100) for l in label] for label in labels["input_ids"]
                ]
            inputs["labels"] = labels["input_ids"]
            return inputs
        else:
            with CaptureLogger(self.tok_logger) as cl:
                inputs = self.tokenizer(
                    inputs, 
                    return_tensors="pt",
                    max_length=self.max_source_length, 
                    padding=self.padding, 
                    truncation=True,
                )
                labels = self.tokenizer(
                    labels, 
                    return_tensors="pt",
                    max_length=self.max_target_length, 
                    padding=self.padding, 
                    truncation=True,
                )
                
            new_inputs = inputs.copy()
            for k, v1 in inputs.items():
                v2 = labels[k]
                new_inputs[k] = torch.cat((v1, v2), dim=1)
                
            new_labels = torch.cat((-100*torch.ones_like(inputs["input_ids"]), labels["input_ids"]), dim=1)
            new_inputs["labels"] = new_labels

            # clm input could be much much longer than block_size
            if "Token indices sequence length is longer than the" in cl.out:
                self.tok_logger.warning(
                    "^^^^^^^^^^^^^^^^ Please ignore the warning above - this long input will be chunked into smaller bits"
                    " before being passed to the model."
                )
            return new_inputs


    def group_texts(self, examples):
        # ['input_ids', 'attention_mask', 'labels']
        concatenated_examples = {k: list(chain(*examples[k])) for k in examples.keys()}
        return concatenated_examples


    def group_ED(self, dataset: Dataset):
        results = {
            'conv_id': [], 
            'prompt': [],
            'dialog': [], 
            'context': [],
        }
        for i, instance in enumerate(dataset):
            if instance['utterance_idx'] == 1:
                results['conv_id'].append(instance['conv_id'])
                results['dialog'].append([])
                results['prompt'].append(instance['prompt'])
                results['context'].append(instance['context'])

            response = {'text': instance['utterance'], 'speaker_idx': instance['speaker_idx']}
            results['dialog'][-1].append(response)
        return Dataset.from_dict(results)


    def preprocess(self, dataset: Dataset):
        if self.dataset == "empathetic_dialogues":
            dataset = self.group_ED(dataset)

        dataset = dataset.map(
            self.tokenize_and_align_labels,
            batched=False,
            num_proc=self.preprocessing_num_workers,
            remove_columns=dataset.column_names,
            load_from_cache_file=not self.overwrite_cache,
        )
        dataset = dataset.map(
            self.group_texts,
            batched=True,
            num_proc=self.preprocessing_num_workers,
            load_from_cache_file=not self.overwrite_cache,
        )
        return dataset

In [9]:
model_path = '../../DG_ckpt/bart'
data_n = 'blended_skill_talk'
tokenizer = AutoTokenizer.from_pretrained(model_path)
sp_token = tokenizer.eos_token
datasets = load_dataset(data_n)
train_dataset = datasets['train']
val_dataset = datasets['validation']
test_dataset = datasets['test']
dg = DGDataset(
    dataset="blended_skill_talk",
    task="seq2seq",
    tokenizer=tokenizer,
)

Found cached dataset blended_skill_talk (/home/yuli/.cache/huggingface/datasets/blended_skill_talk/default/1.0.0/8544e13cbbf2fb9b34157f2e2f28c1539e4f36bf0ef2bd96edd138b4000c5ca1)


  0%|          | 0/3 [00:00<?, ?it/s]

In [10]:
def preprocess(
    dataset: Dataset, 
    tokenizer: Union[BertTokenizer, BartTokenizer, T5Tokenizer, GPT2Tokenizer],
): 
    processed = []
    for i, ins in tqdm(enumerate(dataset)):
        num_entries, total_entries, context, prev_utt_pc = dg.prepare_context(ins)
        for entry_idx in range(num_entries):
            free_message, guided_message, original_context, references = dg.prepare_entry(
                ins, 
                entry_idx, 
                context, 
                prev_utt_pc,
                total_entries,
            )
            if guided_message is None:
                continue
            
            prev_utt_pc += [
                free_message,
                guided_message,
            ]
            
            # Original generation
            text = original_context + sp_token + free_message
            for ref in references:
                processed.append({
                    'src': text,
                    'tgt': ref,
                    'src_len': len(tokenizer.tokenize(text)),
                    'tgt_len': len(tokenizer.tokenize(ref)),
                })
            # processed.append({
            #     'input': text,
            #     'references': references,
            #     'input_length': len(tokenizer.tokenize(text)),
            #     'references_length': [len(tokenizer.tokenize(ref)) for ref in references],
            # })
    processed = pd.DataFrame(processed, columns=['src', 'tgt', 'src_len', 'tgt_len'])
    return processed

data_dir = '../datasets'
if not os.path.exists(data_dir):
    os.makedirs(data_dir)
processed_train = preprocess(train_dataset, tokenizer)
processed_val = preprocess(val_dataset, tokenizer)
processed_test = preprocess(test_dataset, tokenizer)
processed_train.to_csv(f'{data_dir}/train.tsv', sep='\t', index=False)
processed_val.to_csv(f'{data_dir}/val.tsv', sep='\t', index=False)
processed_test.to_csv(f'{data_dir}/dev.tsv', sep='\t', index=False)

4819it [00:48, 99.75it/s] 
1009it [00:10, 92.28it/s]
980it [00:10, 90.73it/s] 


In [19]:
tgt = "i love acting ! i'll be famous someday . what do you do ?"
inputs = tokenizer(tgt, return_tensors="pt", max_length=128, truncation=True)

### Using a light-weight model to predict the length of a sentence

In [20]:
processed_test.groupby('src', as_index=False).agg(list)
# for i, row in :
#     src, tgt, src_len, tgt_len = row['src'], row['tgt'], row['src_len'], row['tgt_len']
#     print(row)

,src,tgt,src_len,tgt_len
0,<PS>basketball is my favorite sport to watch.<...,[i absolutely do not i am sick of politics . i...,"[62, 62, 62, 62]","[15, 8, 11, 19]"
1,<PS>basketball is my favorite sport to watch.<...,[beats spending the day working in the candy s...,"[102, 102, 102, 102]","[18, 14, 20, 22]"
2,<PS>basketball is my favorite sport to watch.<...,[i've been working at a candy store so i just ...,"[145, 145, 145, 145]","[16, 16, 16, 17]"
3,<PS>basketball is my favorite sport to watch.<...,[i've been working at a candy store so i just ...,"[182, 182, 182, 182]","[16, 6, 13, 20]"
4,<PS>basketball is my favorite sport to watch.<...,[good idea ! i keep my mind off things by runn...,"[218, 218, 218, 218]","[12, 10, 14, 16]"
...,...,...,...,...
5477,<PS>zebras are my favorite animals.<SEP>i love...,"[i love lizards , and what a great name !, Nic...","[84, 84, 84, 84]","[11, 24, 28, 21]"
5478,<PS>zebras are my favorite animals.<SEP>i love...,"[nope , just my lizard gila ., I have 3. What ...","[114, 114, 114, 114]","[9, 8, 31, 19]"
5479,<PS>zebras are my favorite animals.<SEP>i love...,[how exciting . how old ? it must be interesti...,"[153, 153, 153, 153]","[15, 21, 31, 31]"
5480,<PS>zebras are my favorite animals.<SEP>i love...,[that's so adorable . anything else you want t...,"[206, 206, 206, 206]","[15, 4, 29, 29]"


In [4]:
from ..seq2seq.models import EncoderRNN, DecoderRNN, Seq2seq

model_path = '../../DG_ckpt/bart'
data_n = 'blended_skill_talk'
tokenizer = AutoTokenizer.from_pretrained(model_path)
max_len = 128
hidden_size = 256
input_dropout = 0
dropout = 0.2
n_layers = 1
bidirectional = True
rnn_cell = 'gru'
attention = False


encoder = EncoderRNN(
    vocab_size=tokenizer.__len__(), 
    max_len=max_len, 
    hidden_size=hidden_size,
    input_dropout_p=input_dropout,
    dropout_p=dropout,
    n_layers=n_layers,
    bidirectional=bidirectional, 
    rnn_cell=rnn_cell,
)
decoder = DecoderRNN(
    vocab_size=tokenizer.__len__(), 
    max_len=max_len, 
    hidden_size=hidden_size * 2 if bidirectional else hidden_size,
    sos_id=tokenizer.bos_token_id,
    eos_id=tokenizer.eos_token_id,
    input_dropout_p=input_dropout,
    dropout_p=dropout,
    n_layers=n_layers,
    bidirectional=bidirectional,
    rnn_cell=rnn_cell,
    use_attention=attention,
)
model = Seq2seq(encoder, decoder)

ModuleNotFoundError: No module named 'seq2seq'